In [1]:
# -*- coding: utf-8 -*-
import warnings
import os.path
import sys
import pickle
import datetime
import pandas as pd
import numpy as np
import xgboost as xgb
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso, Ridge
from sklearn.preprocessing import PolynomialFeatures
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import make_pipeline
import ow_f01, cxmn_train, cxmn_predict, predict_mean
from code.refactor.common import loadSettingsFromYamlFile, save_object, object2Float, get_column_by_type

warnings.filterwarnings("ignore", category=DeprecationWarning)

/home/ubuntu/lijingjie/ljj/local/lib/python2.7/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [2]:
pd.options.display.max_columns=999
pd.options.display.max_rows=999

In [3]:
p1_used_header = ['productkey', 'promotionkey', 'startdatetime', 'enddatetime', 'jdprice', 'syntheticgrossprice', 'promotiondesc', 'promotiondesc_flag', 'promotiontype', 'promotionsubtype',
                'areatypearray', 'tokenflag', 'directdiscount_discount', 'directdiscount_availabilitynumber', 'bundle_subtype1_threshold', 'bundle_subtype1_giveaway',
                'bundle_subtype4_threshold1', 'bundle_subtype4_giveaway1', 'bundle_subtype4_threshold2', 'bundle_subtype4_giveaway2', 'bundle_subtype4_threshold3',
                'bundle_subtype4_giveaway3', 'bundle_subtype2_threshold', 'bundle_subtype2_giveaway', 'bundle_subtype2_maximumgiveaway', 'bundle_subtype15_thresholdnumber1',
                'bundle_subtype15_giveawayrate1', 'bundle_subtype15_thresholdnumber2', 'bundle_subtype15_giveawayrate2', 'bundle_subtype15_thresholdnumber3',
                'bundle_subtype15_giveawayrate3', 'bundle_subtype6_thresholdnumber', 'bundle_subtype6_freenumber', 'suit_maxvaluepool', 'suit_minvaluepool', 'suit_avgvaluepool',
                'suit_discount', 'directdiscount_saleprice', 'bundle_subtype1_percent', 'bundle_subtype4_percent', 'bundle_subtype2_percent', 'bundle_subtype15_percent',
                'bundle_subtype6_percent', 'suit_percent', 'allpercentdiscount', 'mainproductkey', 'hierarchylevel3key', 'createdate', 'statuscode', 'dt']
p2_used_header = ['ProductKey', 'Date', 'HierarchyLevel3Key', 'PromotionCount', 'bundlecount', 'MaxDiscount', 'MinDiscount', 'AvgDiscount', 'MaxSyntheticDiscountA',
				         'MinSyntheticDiscountA', 'AvgSyntheticDiscountA', 'MaxBundleDiscount', 'MinBundleDiscount', 'AvgBundleDiscount', 'MaxDirectDiscount', 'MinDirectDiscount',
				         'AvgDirectDiscount', 'MaxFreegiftDiscount', 'MinFreegiftDiscount', 'AvgFreegiftDiscount', 'SyntheticGrossPrice', 'promotionkey', 'promotiontype',
				         'promotionsubtype', 'syntheticgrossprice_vb', 'jdprice', 'syntheticdiscounta_vb', 'durationinhours', 'daynumberinpromotion', 'bundleflag', 'directdiscountflag',
				         'freegiftflag', 'suitflag', 'numberproducts', 'numberhierarchylevel1', 'numberhierarchylevel2', 'numberhierarchylevel3', 'strongmark', 'stockprice', 'dt']
suffix = '.da'
item = 'p1'
for_what = ['train', 'predict']

In [4]:
def statitics_mape(new_df_final):
		new_df_sku = new_df_final.groupby('ProductKey').sum().reset_index()
		print "ensemble pred sum : %f"%(new_df_sku.ypred_mean_promo_new.sum())
		print "raw pred sum :      %f"%(new_df_sku.ypred_raw.sum())
		print "actual sum:         %f"%(new_df_sku.salesForecast.sum())

		print "raw pred residual:      %f"%(np.sum(np.abs(new_df_sku.ypred_raw - new_df_sku.salesForecast)))
		print "ensemble pred residual: %f"%(np.sum(np.abs(new_df_sku.ypred_mean_promo_new - new_df_sku.salesForecast)))

		print "raw pred mape: %f"%(np.sum(np.abs(new_df_sku.ypred_raw - new_df_sku.salesForecast)) / new_df_sku.salesForecast.sum())
		print "ensemble mape: %f"%(np.sum(np.abs(new_df_sku.ypred_mean_promo_new - new_df_sku.salesForecast)) / new_df_sku.salesForecast.sum())

In [5]:
def get_online_history_data():
    # p1 = pd.read_csv('/home/ubuntu/sunjiadong/promotion_offline/tmp/data/shishang/12029/12029_p1.da',sep='\t',header=None)
    # p2 = pd.read_csv('/home/ubuntu/sunjiadong/promotion_offline/tmp/data/shishang/12029/12029_p2.da',sep='\t',header=None)
    # ts = pd.read_csv('/home/ubuntu/sunjiadong/promotion_offline/tmp/data/shishang/12029/12029_ts.da',sep='\t',header=None)
    # p1 = pd.read_csv('/home/ubuntu/sunjiadong/promotion_offline/tmp/data/7054_p1.da', sep='\t', header=None)
    # p2 = pd.read_csv('/home/ubuntu/sunjiadong/promotion_offline/tmp/data/7054_p2.da', sep='\t', header=None)
    # ts = pd.read_csv('/home/ubuntu/sunjiadong/promotion_offline/tmp/data/7054_ts.da', sep='\t', header=None)
    # p1 = pd.read_csv('/home/ubuntu/sunjiadong/promotion_offline/tmp/data/736_p1.da', sep='\t', header=None)
    # p2 = pd.read_csv('/home/ubuntu/sunjiadong/promotion_offline/tmp/data/736_p2.da', sep='\t', header=None)
    # ts = pd.read_csv('/home/ubuntu/sunjiadong/promotion_offline/tmp/data/736_ts.da', sep='\t', header=None)
    # p1 = pd.read_csv('./diannaobangong/700_p1.csv', sep='\t', header=None)
    # p2 = pd.read_csv('./diannaobangong/700_p2.csv', sep='\t', header=None)
    # ts = pd.read_csv('./diannaobangong/700_ts.csv', sep='\t', header=None)
    p1 = pd.read_csv('./diannaobangong/7371_p1.csv', sep='\t', header=None)
    p2 = pd.read_csv('./diannaobangong/7371_p2.csv', sep='\t', header=None)
    ts = pd.read_csv('./diannaobangong/7371_ts.csv', sep='\t', header=None)
    return p1,p2,ts


In [6]:
scenarioSettingsPath = 'code/refactor/ow_scenario.yaml'
scenario = loadSettingsFromYamlFile(scenarioSettingsPath)
area_rdc_map = pd.read_csv('/home/ubuntu/yulong/promotion_offline/tmp/ow_deploy_single/area_rdc_mapping.csv')
holidays_df=pd.read_csv('/home/ubuntu/yulong/promotion_offline/tmp/ow_deploy_single/holidays.csv')
seasonality_df = pd.read_csv('tmp/data/870_season.csv', parse_dates=['Date'])

In [7]:
cate =  7371 # 7371 700 736
valid_date = pd.to_datetime('2018-04-10')
pred_date = pd.to_datetime('2018-05-08')
scenario['lookforwardPeriodDays'] = 28
# cate = 7054
# valid_date = pd.to_datetime('2018-01-04')
# pred_date = pd.to_datetime('2018-02-01')
# scenario['lookforwardPeriodDays'] = 28

# cate = 12029
# pred_date = pd.to_datetime('2018-05-07')
# scenario['lookforwardPeriodDays'] = 7


In [8]:
p1_df,p2_df,ts_df = get_online_history_data()

/home/ubuntu/lijingjie/ljj/local/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2822: DtypeWarning: Columns (38) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


In [9]:
p1_df.columns = p1_used_header
p2_df.columns = p2_used_header


In [19]:
p1_df.jdprice

0          6.5
1          6.5
2          6.5
3          6.5
4          6.5
5          6.5
6          6.5
7          6.5
8          6.5
9          6.5
10         6.5
11         6.5
12         6.5
13         6.5
14         6.5
15         6.5
16         6.5
17         6.5
18         6.5
19         6.5
20         6.5
21         6.5
22         6.5
23         6.5
24         6.5
25         6.5
26         6.5
27         6.5
28         6.5
29         6.5
30         6.5
31         6.5
32         6.5
33         6.5
34         6.5
35         6.5
36         6.5
37         6.5
38         6.5
39         6.5
40         6.5
41         6.5
42         6.5
43         6.5
44         6.5
45         6.5
46         6.5
47         6.5
48         6.5
49         6.5
50        36.0
51        29.0
52        36.0
53        36.0
54        36.0
55        36.0
56        36.0
57        36.0
58        36.0
59        40.0
60        29.0
61        36.0
62        29.0
63        36.0
64        36.0
65        36.0
66        

In [13]:
p1_df.replace('None', np.nan, inplace=True)
p1_df.replace('null', np.nan, inplace=True)
p1_df.replace(-999, np.nan, inplace=True)
p1_df.replace(' null', np.nan, inplace=True)

In [15]:
raw_df_na = (p1_df.isnull().sum() / len(p1_df)) * 100

raw_df_na = raw_df_na.drop(raw_df_na[raw_df_na == 0].index).sort_values(ascending=False)

missing_data = pd.DataFrame({'Missing Ratio' :raw_df_na})

In [16]:
missing_data

,Missing Ratio
suit_discount,100.000000
suit_avgvaluepool,100.000000
suit_maxvaluepool,100.000000
suit_minvaluepool,100.000000
bundle_subtype1_threshold,99.856837
bundle_subtype1_giveaway,99.856837
bundle_subtype2_maximumgiveaway,99.823810
bundle_subtype6_freenumber,99.645820
bundle_subtype6_thresholdnumber,99.645820
bundle_subtype4_threshold3,96.406412


In [10]:
ts_df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,2018-08-12,0,10,1061010,670,729,7371,54993,0,0.0,0.0,0,0,0,0,0.0,1
1,2018-08-11,1,10,1061010,670,729,7371,54993,0,0.0,0.0,0,0,0,0,0.0,1
2,2018-08-10,2,10,1061010,670,729,7371,54993,0,0.0,0.0,0,0,0,0,0.0,1
3,2018-08-09,3,10,1061010,670,729,7371,54993,0,0.0,0.0,0,0,0,0,0.0,1
4,2018-08-08,4,10,1061010,670,729,7371,54993,0,0.0,0.0,0,0,0,0,0.0,1


In [11]:
print p2_df.Date.min()
print p2_df.Date.max()

2015-01-01
2018-07-09


In [12]:
###handle p2
p2_df['Date'] = pd.to_datetime(p2_df['Date'])
p2_df['dt'] = pd.to_datetime(p2_df['dt'])
p2_df.replace('null', np.nan, inplace=True)
p2_df.replace('None', np.nan, inplace=True)
p2_df.replace(-999, np.nan, inplace=True)
p2_df.drop_duplicates(inplace=True)

# Convert Object -> float
if 'object' in p2_df.dtypes.values:
    obj_cols = get_column_by_type(p2_df,'object')
    object2Float(p2_df,obj_cols)


convert to Float columns:
MaxSyntheticDiscountA
convert to Float columns:
MinSyntheticDiscountA
convert to Float columns:
AvgSyntheticDiscountA
convert to Float columns:
SyntheticGrossPrice
convert to Float columns:
promotionkey
convert to Float columns:
promotiontype
convert to Float columns:
promotionsubtype
convert to Float columns:
syntheticgrossprice_vb
convert to Float columns:
jdprice
convert to Float columns:
syntheticdiscounta_vb
convert to Float columns:
durationinhours
convert to Float columns:
daynumberinpromotion
convert to Float columns:
bundleflag
convert to Float columns:
directdiscountflag
convert to Float columns:
freegiftflag
convert to Float columns:
suitflag
convert to Float columns:
stockprice


In [13]:
def handle_f01(p1_df, for_what, area_rdc_map, pred_date, scenario):
    for fw in for_what:
    	print "output and save: %s_p1_%s"%(str(cate),fw)
    	train_pred_gate = fw   # 'train'
    	f01_output = ow_f01.generate_f01_promo(area_rdc_map, p1_df, scenario, train_pred_gate, ForecastStartDate=pred_date)
    	if train_pred_gate == 'train':
    		train_p1_df = f01_output
    	else:
    		predict_p1_df = f01_output
    return train_p1_df, predict_p1_df


In [14]:
train_p1_df,predict_p1_df = handle_f01(p1_df, for_what, area_rdc_map, pred_date, scenario)

output and save: 7371_p1_train
convert to Float columns:
allpercentdiscount
convert to Float columns:
bundle_subtype15_giveawayrate1
convert to Float columns:
bundle_subtype15_giveawayrate2
convert to Float columns:
bundle_subtype15_giveawayrate3
convert to Float columns:
bundle_subtype15_percent
convert to Float columns:
bundle_subtype15_thresholdnumber1
convert to Float columns:
bundle_subtype15_thresholdnumber2
convert to Float columns:
bundle_subtype15_thresholdnumber3
convert to Float columns:
bundle_subtype1_giveaway
convert to Float columns:
bundle_subtype1_percent
convert to Float columns:
bundle_subtype1_threshold
convert to Float columns:
bundle_subtype2_giveaway
convert to Float columns:
bundle_subtype2_maximumgiveaway
convert to Float columns:
bundle_subtype2_percent
convert to Float columns:
bundle_subtype2_threshold
convert to Float columns:
bundle_subtype4_giveaway1
convert to Float columns:
bundle_subtype4_giveaway2
convert to Float columns:
bundle_subtype4_giveaway3
co

/home/ubuntu/lijingjie/ljj/local/lib/python2.7/site-packages/pandas/core/frame.py:3027: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)
code/refactor/common.py:2111: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  period_promo_raw['rdc_list'] = period_promo_raw['areatypearray'].apply(lambda x: extract_rdc(x, area_rdc_dict))
/home/ubuntu/lijingjie/ljj/local/lib/python2.7/site-packages/pandas/core/generic.py:4355: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.

('| - Before filter: ', 397843)
('| - Filtered tokens: ', 396002)
('| - Filtered descriptions: ', 394267)
('| - Filtered flash sale available units', 394267)
('| - Filtered non-national promo: ', 394256)
| | - Add additional columns
| | - Calculate dd price per promotion
| | - Calculate dd price per period
| | - Generate bundle features
| | - Calculate bundle price per period
| - Reformat date
| | - process features
| | - aggregate features per period
| | - prep data for Step 3
| | - fix indicators on days with end time 00:00 or start time 23:59
| | - calculate weighted price per day
| | - aggregate features per day
f01 shape
(3907645, 23)
output and save: 7371_p1_predict
('| - Before filter: ', 54066)
('| - Filtered tokens: ', 53778)
('| - Filtered descriptions: ', 53594)
('| - Filtered flash sale available units', 53594)
('| - Filtered non-national promo: ', 53594)
| | - Add additional columns
| | - Calculate dd price per promotion
| | - Calculate dd price per period
| | - Generate b

In [15]:
print train_p1_df.shape
print predict_p1_df.shape

(3907645, 23)
(1549160, 23)


In [16]:
print predict_p1_df.Date.max()
print predict_p1_df.Date.min()

2019-01-01 00:00:00
2014-10-01 00:00:00


In [17]:
ts_df.columns = ['Date', 'ind', 'RDCKey', 'ProductKey', 'HierarchyLevel1Key', 'HierarchyLevel2Key', 'HierarchyLevel3Key', 'brand_code', 'sales', 'priceAfterDiscount', 'jd_prc', 'vendibility', 'counterState', 'salesForecast', 'reserveState', 'stockQuantity', 'utc_flag']
ts_df['Date'] = pd.to_datetime(ts_df['Date'])

ts_df.replace('null', np.nan, inplace=True)
ts_df.replace(-999, np.nan, inplace=True)
ts_df.replace('None', np.nan, inplace=True)
ts_to_float_col = ['RDCKey', 'ProductKey', 'HierarchyLevel1Key', 'HierarchyLevel2Key', 'HierarchyLevel3Key', 'brand_code', 'sales', 'priceAfterDiscount', 'jd_prc', 'vendibility', 'counterState', 'salesForecast', 'reserveState', 'stockQuantity', 'utc_flag']
if 'object' in ts_df[ts_to_float_col].dtypes.values:
    object2Float(ts_df,ts_to_float_col)


levels = ['HierarchyLevel3Key','ProductKey','RDCKey','Date']
ts_df.sort_values(levels, ascending=[True]*len(levels), inplace=True)
ts_df= ts_df.reset_index(drop=True)

In [18]:
ts_df_bak = ts_df.copy()

In [19]:
print ts_df.Date.min()
print ts_df.Date.max()

2015-04-18 00:00:00
2018-08-12 00:00:00


In [20]:
# -*- coding: utf-8 -*-
import warnings
import os.path
from code.refactor.common import (get_setting_from_cfg, get_setting_path, filter_df_by_cate_id,get_future_sales_feature,\
                                  dummy_features, model_predict, get_weekly_df,get_future_condition_sales_feature,gen_train_valid_by_date,\
                                  generate_promotion_future, get_hour, save_object, add_pv,get_lowest_n,generate_cv_train_valid_set,\
                                  generate_default_values_by_dic,fill_col_with_default, week2month,trans_band_to_int,get_future_condition_sales_feature_stage2,\
                                  loadSettingsFromYamlFile,createSeasonalityFeatures,createLevel3Features,createSeasonalityDecomposeFeatures,\
                                 calculateNationalRolling,calculateRolling,calculateLagging,createDateFeatures,splitTimeWindow,calculateSimilarRolling,calculateStockFeatures,\
                                 process_rdc,clean_data,add_cols,get_dd_price,agg_dd_price,get_bundle_feat,agg_bd_price,process_feature,\
                                 agg_feature,prep_data,calc_weighted_price,agg_feature_day,get_column_by_type,object2Float,object2Int)
from code.refactor.fdc_flow import  filter_non_price_fill_it
import pandas as pd
import xgboost as xgb
import numpy as np
import datetime
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [21]:
ForecastStartDate = pd.to_datetime(pred_date)
print scenario['lookbackPeriodDays']

400


In [22]:
print ForecastStartDate

2018-05-08 00:00:00


In [23]:
DataStartDate = ForecastStartDate - datetime.timedelta(days=scenario['lookbackPeriodDays'])

In [24]:
print DataStartDate

2017-04-03 00:00:00


In [25]:
holidays_df['Date'] = pd.to_datetime(holidays_df['Date'])
object2Int(holidays_df,['Holiday','Ind_1111_pre','Ind_1111','Ind_1111_post','Ind_618_pre','Ind_618','Ind_618_post','Ind_1212'])
for col in area_rdc_map.columns:
    area_rdc_map[col] = area_rdc_map[col].astype('int')
period_promo_raw = train_p1_df
promoCalendarDf = p2_df

convert to Int columns:
Holiday
convert to Int columns:
Ind_1111_pre
convert to Int columns:
Ind_1111
convert to Int columns:
Ind_1111_post
convert to Int columns:
Ind_618_pre
convert to Int columns:
Ind_618
convert to Int columns:
Ind_618_post
convert to Int columns:
Ind_1212


In [26]:
promoCalendarDf = promoCalendarDf[(promoCalendarDf['Date']<ForecastStartDate)&(promoCalendarDf['Date']>DataStartDate)]

ts_df_pred = ts_df[(ts_df['Date']<ForecastStartDate)&(ts_df['Date']>DataStartDate)]


period_promo_raw.drop_duplicates(inplace=True)
promo_period = period_promo_raw


In [27]:
raw_df= ts_df_pred[ts_df_pred['Date']<ForecastStartDate]


''' initialization'''
if 'ind' in raw_df.columns.values:
    del raw_df['ind']
groupKeys = scenario['selectedColumns']['levelsCoarse'] # ['HierarchyLevel3Key','ProductKey','RDCKey']

In [28]:
min_dt_record = raw_df.groupby(groupKeys)['Date'].min().reset_index(name='dt_min')
min_dt_onstock_record = raw_df[raw_df.counterState==1].groupby(groupKeys)['Date'].min().reset_index(name='dt_onstock_min')
merged_min_dt = pd.merge(min_dt_record,min_dt_onstock_record,on=groupKeys)
valid_merged_min_dt = merged_min_dt[merged_min_dt.dt_min ==merged_min_dt.dt_onstock_min]

raw_df = pd.merge(raw_df, valid_merged_min_dt[groupKeys+['dt_onstock_min']],how='left',on=groupKeys)
raw_df.reset_index(drop=True,inplace=True)
''' fillZeroRows '''
raw_df= filter_non_price_fill_it(raw_df, scenario, price_col_ts='priceAfterDiscount')

In [29]:
q = scenario['quantile'] # 0.95

unitVolumeQuantile = raw_df.groupby(groupKeys)[scenario['Target_col']].quantile(q, interpolation='lower')
quantileDf = pd.DataFrame(unitVolumeQuantile).reset_index()
quantileDf.rename(columns = {scenario['Target_col']: 'UnitCapVolume'}, inplace=True)

# join quantile
raw_df= pd.merge(raw_df, quantileDf, how='left', on=groupKeys)
raw_df['OrderNonOutlierVolume'] = raw_df[['salesForecast', 'UnitCapVolume']].min(axis=1)

# drop cap volume
raw_df.drop(['UnitCapVolume'], axis=1, inplace = True)
if True:
    f = lambda x: x.shift(1).fillna(method='bfill')
    raw_df[['counterState1']] = raw_df[['counterState','ProductKey','RDCKey']].groupby(['ProductKey','RDCKey']).transform(f)
    raw_df[['vendibility1']] = raw_df[['vendibility','ProductKey','RDCKey']].groupby(['ProductKey','RDCKey']).transform(f)

featuresDf = raw_df


In [30]:
featuresDf.Date.max()

Timestamp('2018-05-07 00:00:00')

In [31]:
if seasonality_df is None:
    if (ts_df_pred.Date.max() - ts_df_pred.Date.min()).days > 730:
        print 'running Seasonality'
        featuresDf = createSeasonalityDecomposeFeatures(featuresDf, ['HierarchyLevel3Key', 'ProductKey'], 'sku', scenario)
        featuresDf = createSeasonalityDecomposeFeatures(featuresDf, ['HierarchyLevel3Key'], 'level3', scenario)

        '''
            createLevel3Features
        '''
        featuresDf = createLevel3Features(featuresDf,scenario)
    else:
        print 'faking Seasonality Due to less 2 year Data'
        featuresDf['skuDecomposedTrend'],featuresDf['skuDecomposedSeasonal'],featuresDf['level3DecomposedTrend'],featuresDf['level3DecomposedSeasonal'],featuresDf['Curve'] = [np.nan,np.nan,np.nan,np.nan,np.nan]
else:
    if 'OrderNonOutlierVolume' in seasonality_df.columns:
        seasonality_df.drop(['OrderNonOutlierVolume'], axis=1, inplace = True)
    featuresDf = pd.merge(featuresDf,seasonality_df,on=['HierarchyLevel3Key','ProductKey','Date','RDCKey'],how='left')
'''加入Holiday '''

'\xe5\x8a\xa0\xe5\x85\xa5Holiday '

In [32]:
featuresDf = pd.merge(featuresDf, holidays_df, on='Date', how='left')
'''
    createDateFeatures
'''
featuresDf = createDateFeatures(featuresDf)

'''
    rolling part, 28min, too slow!
'''

featuresDf = pd.merge(featuresDf, promoCalendarDf[promoCalendarDf.columns.difference(['HierarchyLevel1Key','HierarchyLevel2Key', 'HierarchyLevel3Key'])],   left_on = ['ProductKey', 'Date'], right_on = ['ProductKey', 'Date'], how='left')
if scenario['STOCK_PRICE_FLAG']:
    featuresDf = calculateStockFeatures(featuresDf)


featuresDf = calculateNationalRolling(featuresDf, scenario, scenario['selectedColumns']['target'])
featuresDf = calculateRolling(featuresDf, scenario, scenario['selectedColumns']['target'])
featuresDf = calculateSimilarRolling(featuresDf, scenario, scenario['Target_col'])
featuresDf = calculateLagging(featuresDf, scenario, scenario['Target_col'])

featuresDf = pd.merge(featuresDf,  promo_period,on = ['ProductKey', 'Date'], how = 'left')

| Add date features...
| - Add month of the year...
| - Add day of the week...
| Calculating national rolling features...
('| - Rolling value:', 'xxxHashColumn')
| - Rolling 14 mean...
| - Rolling 7 mean...
| - Rolling 5 mean...
| - Rolling 3 mean...
| - Rolling 2 mean...
| - Rolling 1 mean...
| - Rolling 14 median...
| - Rolling 7 median...
| - Rolling decay 28 mean...
| - Rolling decay 14 mean...
| - Rolling decay 7 mean...
| - Rolling decay 3 mean...
| - Added rolling features in 203.977174 seconds 

| Calculating  rolling features...
('| - Rolling value:', 'OrderNonOutlierVolume')
| - Rolling 360 mean...
| - Rolling 180 mean...
| - Rolling 90 mean...
| - Rolling 28 mean...
| - Rolling 14 mean...
| - Rolling 7 mean...
| - Rolling 5 mean...
| - Rolling 3 mean...
| - Rolling 2 mean...
| - Rolling 1 mean...
| - Rolling 14 median...
| - Rolling 7 median...
| - Rolling decay 360 mean...
| - Rolling decay 180 mean...
| - Rolling decay 90 mean...
| - Rolling decay 28 mean...
| - Rolling de

In [33]:
featuresDf.shape

(5900205, 141)

In [34]:
xColumns = scenario['selectedColumns']['features']
yColumns = scenario['selectedColumns']['target']
xColumns.remove('skuDecomposedTrend')
xColumns.remove('skuDecomposedSeasonal')
xColumns.remove('level3DecomposedTrend')
xColumns.remove('level3DecomposedSeasonal')
xColumns.remove('Curve')

In [36]:
'''
exclude_cols = ['MaxBundleDiscount',
 'numberproducts',
 'bd_price_daily_min',
 'MinBundleDiscount',
 'AvgBundleDiscount',
 'MaxFreegiftDiscount',
 'MinFreegiftDiscount',
 'AvgFreegiftDiscount',
 'strongmark',
 'bundle_buy199get100_ind',
 'suit_ind',
 'freegift_ind',
 'suit_discount_new_daily_max']
for col in exclude_cols:
    xColumns.remove(col)


if 'RDCKey' not in xColumns:
    print "go"
    xColumns.append('RDCKey')
'''

In [35]:
import copy
model_1_xColumns = copy.deepcopy(xColumns)
model_1_xColumns.remove('RDCKey')

In [36]:
featuresDf.RDCKey = featuresDf.RDCKey.astype('float')

In [37]:
def train_model(scenario, df_feature):
    param = {
    'objective': "reg:linear",
    'booster': "gbtree",
    'eta' : 0.3, # 0.06, #0.01,
    'min_child_weight':1.0,
    'gamma':0,
    'seed':10,
    'max_depth' :10, #changed from default of 8
    'subsample' :1.0, # 0.7
    'colsample_bytree': 1.0,# 0.7
    # 'silent':1,
    'nthread':4,
    'num_round':  20
    }

    y = df_feature[yColumns].values
    
    X = df_feature[model_1_xColumns]
    xgmat = xgb.DMatrix(X.values, label=y, missing=np.NaN)
    model = xgb.train(param, xgmat, int(param['num_round']))
    return model

In [39]:
train_df_1 = featuresDf[featuresDf.Date<='2018-04-09']
train_df_2 = featuresDf[featuresDf.Date>'2018-04-09']

In [40]:
model_hash = {}
grouped = train_df_1.groupby('RDCKey')
for rdc, group in grouped:
    print 'rdc'
    print rdc
    this_model = train_model(scenario, group)
    model_hash[rdc] = this_model

rdc
3.0
rdc
4.0
rdc
5.0
rdc
6.0
rdc
9.0
rdc
10.0
rdc
316.0
rdc
772.0


In [41]:
grouped = train_df_2.groupby('RDCKey')
result_list = []
for rdc, pred_df in grouped:
    if rdc in model_hash.keys():
        this_model = model_hash[rdc]
    else:
        continue
    
    X_future = pred_df[model_1_xColumns]

    future_xtest = xgb.DMatrix(X_future.values, missing=np.NaN )
    ypred = this_model.predict(future_xtest)
    pred_df['ypred'] =ypred
    pred_df['RDCKey'] = rdc

    ''' Tuning result '''
    lanjie = pred_df[(pred_df.ypred<0)]
    if len(lanjie)>0:
        pred_df.ix[lanjie.index,'ypred'] = 0
    result_list.append(pred_df)
final_result = pd.concat(result_list)

/home/ubuntu/lijingjie/ljj/lib/python2.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]
/home/ubuntu/lijingjie/ljj/lib/python2.7/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/ubuntu/lijingjie/ljj/local/lib/python2.7/site-packages/pandas/core/indexing.py:537: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation

In [47]:
print final_result.shape
print final_result.Date.max()

(585957, 142)
2018-05-07 00:00:00


In [43]:
valid_date

Timestamp('2018-04-10 00:00:00')

In [44]:
def get_actual_sales(ts_df, pred_date, scenario):
	simplified_ts_df = ts_df[ts_df.Date.between(pred_date, pd.to_datetime(pred_date)+\
                       pd.DateOffset(days=scenario['lookforwardPeriodDays']-1))]
	return simplified_ts_df
simplified_ts_df = get_actual_sales(ts_df_bak, valid_date, scenario)


In [45]:
simplified_ts_df.shape

(916805, 17)

In [46]:
simplified_ts_df.Date.max()

Timestamp('2018-05-07 00:00:00')

In [48]:
list_keys = ['Date','RDCKey','ProductKey']
df = pd.merge(final_result, simplified_ts_df[list_keys + ['sales','salesForecast']], how='left', on=list_keys)

In [49]:
df.shape

(585957, 144)

In [50]:
df.head()

,Date,RDCKey,ProductKey,HierarchyLevel1Key,HierarchyLevel2Key,HierarchyLevel3Key,brand_code,sales_x,priceAfterDiscount,jd_prc,vendibility,counterState,salesForecast_x,reserveState,stockQuantity,utc_flag,dt_onstock_min,OrderNonOutlierVolume,counterState1,vendibility1,skuDecomposedSeasonal,skuDecomposedTrend,level3DecomposedTrend,level3DecomposedSeasonal,Curve,Holiday,Ind_1111_pre,Ind_1111,Ind_1111_post,Ind_618_pre,Ind_618,Ind_618_post,Ind_1212,Month,DayOfWeek,AvgBundleDiscount,AvgDirectDiscount,AvgDiscount,AvgFreegiftDiscount,AvgSyntheticDiscountA,MaxBundleDiscount,MaxDirectDiscount,MaxDiscount,MaxFreegiftDiscount,MaxSyntheticDiscountA,MinBundleDiscount,MinDirectDiscount,MinDiscount,MinFreegiftDiscount,MinSyntheticDiscountA,PromotionCount,SyntheticGrossPrice,bundlecount,bundleflag,daynumberinpromotion,directdiscountflag,dt,durationinhours,freegiftflag,jdprice,numberhierarchylevel1,numberhierarchylevel2,numberhierarchylevel3,numberproducts,promotionkey,promotionsubtype,promotiontype,stockprice,strongmark,suitflag,syntheticdiscounta_vb,syntheticgrossprice_vb,national_rolling14Mean,national_rolling7Mean,national_rolling5Mean,national_rolling3Mean,national_rolling2Mean,national_rolling1Mean,national_rolling14Median,national_rolling7Median,national_rolling28DecayMean,national_rolling14DecayMean,national_rolling7DecayMean,national_rolling3DecayMean,rolling360Mean,rolling180Mean,rolling90Mean,rolling28Mean,rolling14Mean,rolling7Mean,rolling5Mean,rolling3Mean,rolling2Mean,rolling1Mean,rolling14Median,rolling7Median,rolling360DecayMean,rolling180DecayMean,rolling90DecayMean,rolling28DecayMean,rolling14DecayMean,rolling7DecayMean,rolling3DecayMean,similar_rolling14Mean,similar_rolling7Mean,similar_rolling5Mean,similar_rolling3Mean,similar_rolling2Mean,similar_rolling1Mean,similar_rolling14Median,similar_rolling7Median,similar_rolling28DecayMean,similar_rolling14DecayMean,similar_rolling7DecayMean,similar_rolling3DecayMean,volume_rolling365Shift,volume_rolling180Shift,volume_rolling90Shift,volume_rolling30Shift,volume_rolling7Shift,dd_price_weighted,bd_price_weighted,flashsale_ind,dd_ind,bundle_ind,bundle_buy199get100_ind,suit_ind,freegift_ind,cnt_period,dd_discount_daily_max,dd_discount_sgp_daily_max,bd_discount_daily_max,suit_discount_new_daily_max,dd_price_daily_min,bd_price_daily_min,jdprice_daily_min,sgp_daily_min,dd_discount_wgt,dd_discount_sgp_wgt,bd_discount_wgt,bd_discount_sgp_wgt,ypred,sales_y,salesForecast_y
0,2018-04-10,3,247046,670,729,7371,15097,4,27.5,32.5,1,1,4,0,2.0,0,2017-04-04,4,1.0,1.0,NaN,NaN,NaN,NaN,NaN,3,0,0,0,0,0,0,0,4,1,0.30303,0.0,0.30303,0.0,0.303,0.30303,0.0,0.30303,0.0,0.303,0.30303,0.0,0.30303,0.0,0.303,1.0,32.5,1.0,1.0,6.0,0.0,2018-04-10,24.0,0.0,32.5,1.0,1.0,1.0,243.0,215991190.0,4.0,10.0,22.27,0.0,0.0,0.303,32.5,7.071429,4.857143,4.2,5.666667,6.0,8.0,6.0,4.0,5.526584,5.657482,5.543321,6.021598,1.683333,1.844444,1.155556,0.964286,1.214286,0.571429,0.4,0.333333,0.5,1.0,1.0,1.0,1.612586,1.475880,1.228895,0.984548,0.907902,0.735743,0.609998,NaN,NaN,0.4,0.333333,0.5,1.0,NaN,NaN,0.414813,0.430734,0.466069,0.580645,2.0,1.0,2.0,0.0,1.0,29.0,19.2685,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.107692,0.107692,0.407124,0.0,29.0,19.268456,29.0,32.5,0.0,0.1077,0.3356,0.4071,1.203683,4,4
1,2018-04-11,3,247046,670,729,7371,15097,0,27.5,0.0,1,1,0,0,11.0,0,2017-04-04,0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,2,1,0,0,0,0,0,0,4,2,0.30303,0.0,0.30303,0.0,0.303,0.30303,0.0,0.30303,0.0,0.303,0.30303,0.0,0.30303,0.0,0.303,1.0,32.5,1.0,1.0,7.0,0.0,2018-04-11,24.0,0.0,32.5,1.0,1.0,1.0,243.0,215991190.0,4.0,10.0,22.28,0.0,0.0,0.303,32.5,7.571429,6.428571,7.8,10.666667,14.0,20.0,6.0,4.0,6.524750,7.569817,9.157490,13.010799,1.694444,1.861111,1.177778,1.035714,1.357143,1.000000,1.2,1.666667,2.5,4.0,1.0,1.0,1.627730,1.504226,1.289814,1.192511,1.320181,1.551807,2.304999,NaN,NaN,1.2,1.666667,2.5,4.0,NaN,NaN,1.123928,1.256598,1.540838,2.317460,1.0,0.0,6.0,0.0,1.0,29.0,19.2685,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.107692,0.107692,0.407124,0.0,29.0,19.268456,29.0,32.5,0.0,0

In [51]:
ForecastStartDate = pd.to_datetime(pred_date)

In [52]:
ts_df = ts_df_bak.copy()
print ts_df.Date.max()

2018-08-12 00:00:00


In [53]:
print ForecastStartDate

2018-05-08 00:00:00


In [54]:
PredictEndDate = ForecastStartDate + datetime.timedelta(days=(scenario['lookforwardPeriodDays']-1))
DataStartDate = ForecastStartDate - datetime.timedelta(days=scenario['lookbackPeriodDays_predict'])


print PredictEndDate
print scenario['lookbackPeriodDays_predict']

ts_df = ts_df[(ts_df['Date']<ForecastStartDate)&(ts_df['Date']>DataStartDate)]

holidays_df['Date'] = pd.to_datetime(holidays_df['Date'])
object2Int(holidays_df,['Holiday','Ind_1111_pre','Ind_1111','Ind_1111_post','Ind_618_pre','Ind_618','Ind_618_post','Ind_1212'])
for col in area_rdc_map.columns:
    area_rdc_map[col] = area_rdc_map[col].astype('int')
period_promo_raw = predict_p1_df
promoCalendarDf = p2_df

promoCalendarDf = promoCalendarDf[(promoCalendarDf['Date']<PredictEndDate)&(promoCalendarDf['Date']>DataStartDate)]


period_promo_raw.drop_duplicates(inplace=True)
promo_period = period_promo_raw


''' f02 predict'''

raw_df= ts_df[ts_df['Date']<ForecastStartDate]

if 'ind' in raw_df.columns.values:
    del raw_df['ind']
groupKeys = scenario['selectedColumns']['levelsCoarse'] # ['HierarchyLevel3Key','ProductKey','RDCKey']

''' 获取新品的上柜时间 'dt_onstaock_min'. nan if 上柜时间小于ts start_time '''
min_dt_record = raw_df.groupby(groupKeys)['Date'].min().reset_index(name='dt_min')
min_dt_onstock_record = raw_df[raw_df.counterState==1].groupby(groupKeys)['Date'].min().reset_index(name='dt_onstock_min')
merged_min_dt = pd.merge(min_dt_record,min_dt_onstock_record,on=groupKeys)
valid_merged_min_dt = merged_min_dt[merged_min_dt.dt_min ==merged_min_dt.dt_onstock_min]

raw_df = pd.merge(raw_df, valid_merged_min_dt[groupKeys+['dt_onstock_min']],how='left',on=groupKeys)
raw_df.reset_index(drop=True,inplace=True)
''' fillZeroRows '''
raw_df= filter_non_price_fill_it(raw_df, scenario, price_col_ts='priceAfterDiscount')

q = scenario['quantile'] # 0.95
# q = 0.9

unitVolumeQuantile = raw_df.groupby(groupKeys)[scenario['Target_col']].quantile(q, interpolation='lower')
quantileDf = pd.DataFrame(unitVolumeQuantile).reset_index()
quantileDf.rename(columns = {scenario['Target_col']: 'UnitCapVolume'}, inplace=True)

# join quantile
raw_df= pd.merge(raw_df, quantileDf, how='left', on=groupKeys)
raw_df['OrderNonOutlierVolume'] = raw_df[['salesForecast', 'UnitCapVolume']].min(axis=1)

# drop cap volume
raw_df.drop(['UnitCapVolume'], axis=1, inplace = True)
if True:
    f = lambda x: x.shift(1).fillna(method='bfill')
    raw_df[['counterState1']] = raw_df[['counterState','ProductKey','RDCKey']].groupby(['ProductKey','RDCKey']).transform(f)
    raw_df[['vendibility1']] = raw_df[['vendibility','ProductKey','RDCKey']].groupby(['ProductKey','RDCKey']).transform(f)

df_past = raw_df

def return_nearest_date_index():
    index = [i for i in range(1,10) if (ForecastStartDate- pd.DateOffset(days=i)).strftime('%Y-%m-%d') not in scenario['Remove_by_stat_date']]
    return index[0]

fut_train_end = ForecastStartDate- pd.DateOffset(days=1)
# make sure train end date not in remove date
if fut_train_end.strftime('%Y-%m-%d') in scenario['Remove_by_stat_date']:
    i = return_nearest_date_index()

    fut_train_end = ForecastStartDate- pd.DateOffset(days=i)

''' concat Future Df by RDC'''
dfs=[]
for rdc in list(set(df_past.RDCKey)):

    valid_skus_fut=list(set(df_past[(df_past['Date']==fut_train_end)&(df_past.RDCKey==rdc)]['ProductKey'].values))

    # generate_default_df
    def generate_strutured_null_df(data_start_date, data_end_date, valid_skus, sku_name='ProductKey', date_name='Date'):
        rng = pd.date_range(start=data_start_date, end=data_end_date, freq='D')
        valid_date_span_length = len(rng)
        sku_col = []
        rng_col = []
        for sku_id in valid_skus:
            sku_col.extend(np.repeat(sku_id, valid_date_span_length))
            rng_col.extend(rng)

        dic = {sku_name : sku_col, date_name: rng_col}

        return pd.DataFrame(dic)

    fut_null_df = generate_strutured_null_df(ForecastStartDate, PredictEndDate, valid_skus_fut)  # Filling 0601-0604 cols [stat_date, sku]
    fill_one_cols = {'vendibility1': 1}

    basic_col=['ProductKey','Date','RDCKey','HierarchyLevel1Key','HierarchyLevel2Key','HierarchyLevel3Key'\
            ,'dt_onstock_min','brand_code','salesForecast','reserveState','OrderNonOutlierVolume','vendibility1']
    to_fillna_col=['dt_onstock_min','RDCKey','brand_code','reserveState','HierarchyLevel1Key','HierarchyLevel2Key'\
            ,'HierarchyLevel3Key','vendibility1']

    default_kv = generate_default_values_by_dic(fill_nan_list=to_fillna_col, fill_default_kv=fill_one_cols)

    fut_null_df = fill_col_with_default(fut_null_df, default_kv)

    df_future = df_past[df_past.RDCKey==rdc][basic_col].append(fut_null_df)
    df_future.sort_values(['ProductKey', 'Date'], inplace=True)
    df_future.reset_index(drop=True,inplace=True)
    col = list(df_future.columns.values)
    col.remove('ProductKey')

    df_future[col] = df_future.groupby(['ProductKey']).transform(lambda x: x.fillna(method='ffill'))
    dfs.append(df_future)
df_with_future=pd.concat(dfs)
df_with_future.reset_index(drop=True,inplace=True)

featuresDf = df_with_future

if seasonality_df is None:
    if (ts_df.Date.max() - ts_df.Date.min()).days > 730:
        print 'running Seasonality'
        featuresDf = createSeasonalityDecomposeFeatures(featuresDf, ['HierarchyLevel3Key', 'ProductKey'], 'sku', scenario)
        featuresDf = createSeasonalityDecomposeFeatures(featuresDf, ['HierarchyLevel3Key'], 'level3', scenario)

        '''
            createLevel3Features
        '''
        featuresDf = createLevel3Features(featuresDf,scenario)
    else:
        print 'faking Seasonality Due to less 2 year Data'
        featuresDf['skuDecomposedTrend'],featuresDf['skuDecomposedSeasonal'],featuresDf['level3DecomposedTrend'],featuresDf['level3DecomposedSeasonal'],featuresDf['Curve'] = [np.nan,np.nan,np.nan,np.nan,np.nan]
else:
    if 'OrderNonOutlierVolume' in seasonality_df.columns:
        seasonality_df.drop(['OrderNonOutlierVolume'], axis=1, inplace = True)
    featuresDf = pd.merge(featuresDf,seasonality_df,on=['HierarchyLevel3Key','ProductKey','Date','RDCKey'],how='left')

# -*- coding: utf-8 -*-
import warnings
import os.path
from code.refactor.common import (get_setting_from_cfg, get_setting_path, filter_df_by_cate_id,get_future_sales_feature,\
                                  dummy_features, model_predict, get_weekly_df,get_future_condition_sales_feature,gen_train_valid_by_date,\
                                  generate_promotion_future, get_hour, save_object, add_pv,get_lowest_n,generate_cv_train_valid_set,\
                                  generate_default_values_by_dic,fill_col_with_default, week2month,trans_band_to_int,get_future_condition_sales_feature_stage2,\
                                  loadSettingsFromYamlFile,createSeasonalityFeatures,createLevel3Features,createSeasonalityDecomposeFeatures,\
                                 calculateNationalRolling_predict,calculateRolling_predict,calculateLagging_predict,createDateFeatures,splitTimeWindow,calculateSimilarRolling_predict,calculateStockFeatures,\
                                 process_rdc,clean_data,add_cols,get_dd_price,agg_dd_price,get_bundle_feat,agg_bd_price,process_feature,\
                                 agg_feature,prep_data,calc_weighted_price,agg_feature_day,get_column_by_type,object2Float,object2Int)
from code.refactor.fdc_flow import  filter_non_price_fill_it
import pandas as pd
import xgboost as xgb
import numpy as np
import datetime
import pickle

featuresDf = pd.merge(featuresDf, holidays_df, on='Date', how='left')

'''
    createDateFeatures
'''
featuresDf = createDateFeatures(featuresDf)

'''
    rolling part, 28min, too slow!
'''

# featuresDf = pd.merge(featuresDf, promoCalendarDf[promoCalendarDf.columns.difference(['HierarchyLevel1Key','HierarchyLevel2Key', 'HierarchyLevel3Key'])],   left_on = ['ProductKey', 'Date'], right_on = ['ProductKey', 'Date'], how='left')
featuresDf = pd.merge(featuresDf, promoCalendarDf[promoCalendarDf.columns.difference(['HierarchyLevel1Key','HierarchyLevel2Key', 'HierarchyLevel3Key'])].drop_duplicates(),   on = ['ProductKey', 'Date'], how='left')
if scenario['STOCK_PRICE_FLAG']:
    featuresDf = calculateStockFeatures(featuresDf)

df_fut = featuresDf[featuresDf.Date>=pd.to_datetime(ForecastStartDate)]
# df_fut = featuresDf[featuresDf.Date>=pd.to_datetime('2017-10-01')]
df_past = featuresDf[featuresDf.Date<pd.to_datetime(ForecastStartDate)]
####notice: use selected promotion cols' mean values to fill the same cols in df_future

df_fut = calculateNationalRolling_predict(df_fut,df_past, scenario, scenario['selectedColumns']['target'])
df_fut = calculateRolling_predict(df_fut,df_past, scenario, scenario['selectedColumns']['target'])
df_fut = calculateSimilarRolling_predict(df_fut,df_past, scenario, scenario['Target_col'])
df_fut = calculateLagging_predict(df_fut,df_past, scenario, scenario['Target_col'])

# promo_period.read_csv('f01.csv',parse_dates=['Date'])
df_fut = pd.merge(df_fut,  promo_period,on = ['ProductKey', 'Date'], how = 'left')

2018-06-04 00:00:00
400
convert to Int columns:
Holiday
convert to Int columns:
Ind_1111_pre
convert to Int columns:
Ind_1111
convert to Int columns:
Ind_1111_post
convert to Int columns:
Ind_618_pre
convert to Int columns:
Ind_618
convert to Int columns:
Ind_618_post
convert to Int columns:
Ind_1212
| Add date features...
| - Add month of the year...
| - Add day of the week...


code/refactor/common.py:1363: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  + df_past[similarLevels[2]]
code/refactor/common.py:1364: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df_past['SyntheticPromotionSubType'] = df_past['SyntheticPromotionSubType'].fillna(0.0)


| - group by SyntheticPromotionSubType...


In [95]:
grouped = df_fut.groupby('RDCKey')
result_list = []
for rdc, pred_df in grouped:
    if rdc in model_hash.keys():
        this_model = model_hash[rdc]
    else:
        continue
    
    X_future = pred_df[model_1_xColumns]

    future_xtest = xgb.DMatrix(X_future.values, missing=np.NaN )
    ypred = this_model.predict(future_xtest)
    pred_df['ypred'] =ypred
    pred_df['RDCKey'] = rdc

    ''' Tuning result '''
    lanjie = pred_df[(pred_df.ypred<0)]
    if len(lanjie)>0:
        pred_df.ix[lanjie.index,'ypred'] = 0
    result_list.append(pred_df)
final_result = pd.concat(result_list)

/home/ubuntu/lijingjie/ljj/lib/python2.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]
/home/ubuntu/lijingjie/ljj/lib/python2.7/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [96]:
#print final_result.head()
print final_result.shape
print final_result.Date.max()

(601468, 135)
2018-06-04 00:00:00


In [97]:
test_lgb = final_result.copy()

In [57]:
df.head()

,Date,RDCKey,ProductKey,HierarchyLevel1Key,HierarchyLevel2Key,HierarchyLevel3Key,brand_code,sales_x,priceAfterDiscount,jd_prc,vendibility,counterState,salesForecast_x,reserveState,stockQuantity,utc_flag,dt_onstock_min,OrderNonOutlierVolume,counterState1,vendibility1,skuDecomposedSeasonal,skuDecomposedTrend,level3DecomposedTrend,level3DecomposedSeasonal,Curve,Holiday,Ind_1111_pre,Ind_1111,Ind_1111_post,Ind_618_pre,Ind_618,Ind_618_post,Ind_1212,Month,DayOfWeek,AvgBundleDiscount,AvgDirectDiscount,AvgDiscount,AvgFreegiftDiscount,AvgSyntheticDiscountA,MaxBundleDiscount,MaxDirectDiscount,MaxDiscount,MaxFreegiftDiscount,MaxSyntheticDiscountA,MinBundleDiscount,MinDirectDiscount,MinDiscount,MinFreegiftDiscount,MinSyntheticDiscountA,PromotionCount,SyntheticGrossPrice,bundlecount,bundleflag,daynumberinpromotion,directdiscountflag,dt,durationinhours,freegiftflag,jdprice,numberhierarchylevel1,numberhierarchylevel2,numberhierarchylevel3,numberproducts,promotionkey,promotionsubtype,promotiontype,stockprice,strongmark,suitflag,syntheticdiscounta_vb,syntheticgrossprice_vb,national_rolling14Mean,national_rolling7Mean,national_rolling5Mean,national_rolling3Mean,national_rolling2Mean,national_rolling1Mean,national_rolling14Median,national_rolling7Median,national_rolling28DecayMean,national_rolling14DecayMean,national_rolling7DecayMean,national_rolling3DecayMean,rolling360Mean,rolling180Mean,rolling90Mean,rolling28Mean,rolling14Mean,rolling7Mean,rolling5Mean,rolling3Mean,rolling2Mean,rolling1Mean,rolling14Median,rolling7Median,rolling360DecayMean,rolling180DecayMean,rolling90DecayMean,rolling28DecayMean,rolling14DecayMean,rolling7DecayMean,rolling3DecayMean,similar_rolling14Mean,similar_rolling7Mean,similar_rolling5Mean,similar_rolling3Mean,similar_rolling2Mean,similar_rolling1Mean,similar_rolling14Median,similar_rolling7Median,similar_rolling28DecayMean,similar_rolling14DecayMean,similar_rolling7DecayMean,similar_rolling3DecayMean,volume_rolling365Shift,volume_rolling180Shift,volume_rolling90Shift,volume_rolling30Shift,volume_rolling7Shift,dd_price_weighted,bd_price_weighted,flashsale_ind,dd_ind,bundle_ind,bundle_buy199get100_ind,suit_ind,freegift_ind,cnt_period,dd_discount_daily_max,dd_discount_sgp_daily_max,bd_discount_daily_max,suit_discount_new_daily_max,dd_price_daily_min,bd_price_daily_min,jdprice_daily_min,sgp_daily_min,dd_discount_wgt,dd_discount_sgp_wgt,bd_discount_wgt,bd_discount_sgp_wgt,ypred,sales_y,salesForecast_y
0,2018-04-10,3,247046,670,729,7371,15097,4,27.5,32.5,1,1,4,0,2.0,0,2017-04-04,4,1.0,1.0,NaN,NaN,NaN,NaN,NaN,3,0,0,0,0,0,0,0,4,1,0.30303,0.0,0.30303,0.0,0.303,0.30303,0.0,0.30303,0.0,0.303,0.30303,0.0,0.30303,0.0,0.303,1.0,32.5,1.0,1.0,6.0,0.0,2018-04-10,24.0,0.0,32.5,1.0,1.0,1.0,243.0,215991190.0,4.0,10.0,22.27,0.0,0.0,0.303,32.5,7.071429,4.857143,4.2,5.666667,6.0,8.0,6.0,4.0,5.526584,5.657482,5.543321,6.021598,1.683333,1.844444,1.155556,0.964286,1.214286,0.571429,0.4,0.333333,0.5,1.0,1.0,1.0,1.612586,1.475880,1.228895,0.984548,0.907902,0.735743,0.609998,NaN,NaN,0.4,0.333333,0.5,1.0,NaN,NaN,0.414813,0.430734,0.466069,0.580645,2.0,1.0,2.0,0.0,1.0,29.0,19.2685,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.107692,0.107692,0.407124,0.0,29.0,19.268456,29.0,32.5,0.0,0.1077,0.3356,0.4071,1.203683,4,4
1,2018-04-11,3,247046,670,729,7371,15097,0,27.5,0.0,1,1,0,0,11.0,0,2017-04-04,0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,2,1,0,0,0,0,0,0,4,2,0.30303,0.0,0.30303,0.0,0.303,0.30303,0.0,0.30303,0.0,0.303,0.30303,0.0,0.30303,0.0,0.303,1.0,32.5,1.0,1.0,7.0,0.0,2018-04-11,24.0,0.0,32.5,1.0,1.0,1.0,243.0,215991190.0,4.0,10.0,22.28,0.0,0.0,0.303,32.5,7.571429,6.428571,7.8,10.666667,14.0,20.0,6.0,4.0,6.524750,7.569817,9.157490,13.010799,1.694444,1.861111,1.177778,1.035714,1.357143,1.000000,1.2,1.666667,2.5,4.0,1.0,1.0,1.627730,1.504226,1.289814,1.192511,1.320181,1.551807,2.304999,NaN,NaN,1.2,1.666667,2.5,4.0,NaN,NaN,1.123928,1.256598,1.540838,2.317460,1.0,0.0,6.0,0.0,1.0,29.0,19.2685,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.107692,0.107692,0.407124,0.0,29.0,19.268456,29.0,32.5,0.0,0

In [59]:
train_df = df.copy()

model2_xColumns = copy.deepcopy(xColumns)
len(model2_xColumns)

101

In [88]:
model2_xColumns

['Holiday',
 'Ind_1111_pre',
 'Ind_1111',
 'Ind_1111_post',
 'Ind_618_pre',
 'Ind_618',
 'Ind_618_post',
 'Ind_1212',
 'Month',
 'DayOfWeek',
 'RDCKey',
 'PromotionCount',
 'MaxDiscount',
 'MinDiscount',
 'AvgDiscount',
 'MaxSyntheticDiscountA',
 'MinSyntheticDiscountA',
 'AvgSyntheticDiscountA',
 'MaxBundleDiscount',
 'MinBundleDiscount',
 'AvgBundleDiscount',
 'MaxDirectDiscount',
 'MinDirectDiscount',
 'AvgDirectDiscount',
 'MaxFreegiftDiscount',
 'MinFreegiftDiscount',
 'AvgFreegiftDiscount',
 'SyntheticGrossPrice',
 'syntheticdiscounta_vb',
 'durationinhours',
 'daynumberinpromotion',
 'numberproducts',
 'strongmark',
 'rolling360Mean',
 'rolling180Mean',
 'rolling90Mean',
 'rolling28Mean',
 'rolling14Mean',
 'rolling7Mean',
 'rolling5Mean',
 'rolling3Mean',
 'rolling2Mean',
 'rolling1Mean',
 'rolling14Median',
 'rolling7Median',
 'rolling360DecayMean',
 'rolling180DecayMean',
 'rolling90DecayMean',
 'rolling28DecayMean',
 'rolling14DecayMean',
 'rolling7DecayMean',
 'rolling3Deca

In [89]:
rolling_cols = ['rolling14Mean',
 'rolling7Mean',
 'rolling5Mean',
 'rolling3Mean',
 'rolling2Mean',
 'rolling1Mean',
 'rolling14Median',
 'rolling7Median',
 'rolling14DecayMean',
 'rolling7DecayMean',
 'rolling3DecayMean',
 'national_rolling14Mean',
 'national_rolling7Mean',
 'national_rolling5Mean',
 'national_rolling3Mean',
 'national_rolling2Mean',
 'national_rolling1Mean',
 'national_rolling14Median',
 'national_rolling7Median',
 'national_rolling14DecayMean',
 'national_rolling7DecayMean',
 'national_rolling3DecayMean',
 'similar_rolling14Mean',
 'similar_rolling7Mean',
 'similar_rolling5Mean',
 'similar_rolling3Mean',
 'similar_rolling2Mean',
 'similar_rolling1Mean',
 'similar_rolling14Median',
 'similar_rolling7Median',
 'similar_rolling14DecayMean',
 'similar_rolling7DecayMean',
 'similar_rolling3DecayMean',
 'volume_rolling7Shift']

In [90]:
for col in rolling_cols:
    model2_xColumns.remove(col)

In [91]:
model2_xColumns.append('ypred')

In [92]:
len(model2_xColumns)

69

In [93]:
import lightgbm as lgb
def compute_mapd(reals, predicts):
    total_reals = np.nansum(reals)
    total_abs_diff = np.nansum(np.abs(reals - predicts))
    return total_abs_diff if total_reals == 0.0 else total_abs_diff / total_reals
def error_weighted_mapd_lgb(preds, dtrain):
    labels =  dtrain.get_label()
    return 'wmapd',compute_mapd(labels,preds),False

train_df.RDCKey = train_df.RDCKey.astype(float)
y = train_df['salesForecast_y'].values    
X = train_df[model2_xColumns]
dTrain = lgb.Dataset(X, label=y)

params = {'task': 'train',\
          'boosting_type': 'gbdt',\
          'objective': 'regression',\
          'metric': {'rmse'},\
          'num_leaves': 100}

num_round = 200
model = lgb.train(params, lgb_train, num_boost_round = num_round)
# preds_
#train = model.predict(dataFeaturesTrain,num_iteration=model.best_iteration)
# mapd = compute_mapd(dataLabelsTrain,preds_train)
# preds = model.predict(dataFeaturesValid,num_iteration=model.best_iteration)
# preds = np.array([max(0.0,p) for p in preds])


print model.feature_importance()
print model.feature_name()

[ 145   40    0    0    3    0    0    0  161  868  570  122  350  352
  204  184  199  123   89   17    6  110  105   38    0    0    0  409
   59   42  212   95    0  455  387  588 1161  168  113  125  252  373
  465 2017 1950 2054 2473  188  189   45   63   39    0    0    0  152
   63  139    0   33   60  172  112  169  150  189  221  732    0]
[u'Holiday', u'Ind_1111_pre', u'Ind_1111', u'Ind_1111_post', u'Ind_618_pre', u'Ind_618', u'Ind_618_post', u'Ind_1212', u'Month', u'DayOfWeek', u'RDCKey', u'PromotionCount', u'MaxDiscount', u'MinDiscount', u'AvgDiscount', u'MaxSyntheticDiscountA', u'MinSyntheticDiscountA', u'AvgSyntheticDiscountA', u'MaxBundleDiscount', u'MinBundleDiscount', u'AvgBundleDiscount', u'MaxDirectDiscount', u'MinDirectDiscount', u'AvgDirectDiscount', u'MaxFreegiftDiscount', u'MinFreegiftDiscount', u'AvgFreegiftDiscount', u'SyntheticGrossPrice', u'syntheticdiscounta_vb', u'durationinhours', u'daynumberinpromotion', u'numberproducts', u'strongmark', u'rolling360Mean'

In [94]:
preds_train = model.predict(X)
mapd = compute_mapd(y,preds_train)
print mapd

0.6291756905612585


In [98]:
test_df = test_lgb

In [99]:
test_df.RDCKey = test_df.RDCKey.astype(float)
X_test = test_df[model2_xColumns]
preds_test = model.predict(X_test)
preds = np.array([max(0.0,p) for p in preds_test])

In [100]:
print len(preds)
print test_df.shape

601468
(601468, 135)


In [72]:
test_df.ypred.head(20)

0     19.469112
1     19.469112
2     19.469112
3     19.469112
4     19.837534
5     21.548492
6     19.469112
7     20.946646
8     20.879284
9     20.879284
10    20.946646
11    21.247705
12    21.247705
13    20.946646
14    20.879284
15    20.946646
16    20.879284
17    20.879284
18    21.247705
19    21.247705
Name: ypred, dtype: float32

In [101]:
test_df['model_2_ypred'] = preds

In [102]:
simplified_ts_df_final = get_actual_sales(ts_df_bak, pred_date, scenario)

In [103]:
list_keys = ['Date','RDCKey','ProductKey']
final_result = pd.merge(test_df[list_keys + ['ypred','model_2_ypred']], simplified_ts_df_final[list_keys + ['sales','salesForecast']], how='left', on=list_keys)

In [104]:
final_result.head()

,Date,RDCKey,ProductKey,ypred,model_2_ypred,sales,salesForecast
0,2018-05-08,3,247046,1.097878,1.129269,0,0
1,2018-05-09,3,247046,1.048624,1.144461,3,3
2,2018-05-10,3,247046,1.048624,1.170498,0,0
3,2018-05-11,3,247046,0.994609,1.031739,2,2
4,2018-05-12,3,247046,0.861172,0.811247,0,0


In [68]:
# final_result.to_csv('./result_670/transfer_test_result_736_0508-step28.csv',index=False)
# final_result.to_csv('./result_670/transfer_test_result_7054_0201-step10.csv',index=False)
# final_result.to_csv('./result_670/transfer_test_result_700_0508-step28.csv',index=False)
final_result.to_csv('./result_670/transfer_test_result_7371_0508-step28.csv',index=False)

In [106]:
list_keys = ['Date','RDCKey','ProductKey']
tmp_result = pd.merge(test_df[list_keys + ['ypred','model_2_ypred']], simplified_ts_df_final[list_keys + ['sales','salesForecast']], how='left', on=list_keys)
dd = tmp_result[tmp_result.Date<='2018-06-04']
dd1 = dd.groupby(['RDCKey','ProductKey']).sum().reset_index()
print np.sum(np.abs(dd1.model_2_ypred - dd1.salesForecast)) / np.sum(dd1.salesForecast)

0.34799663742785475


In [74]:
#raw_pred = pd.read_csv('./736_result_0508_new.csv')
#raw_pred = pd.read_csv('./result_670/raw_pred_result_7054_0201-step28.csv')
#raw_pred = pd.read_csv('./result_670/raw_pred_result_700_0508-step28.csv')
raw_pred = pd.read_csv('./result_670/raw_pred_result_7371_0508-step28.csv')

In [75]:
raw_pred = raw_pred[list_keys + ['ypred','salesForecast_y']].rename(columns={"ypred":"ypred_raw"})

In [76]:
raw_pred.Date = pd.to_datetime(raw_pred.Date)

In [77]:
raw_pred.head()

,Date,RDCKey,ProductKey,ypred_raw,salesForecast_y
0,2018-05-08,3.0,247046,1.178358,0
1,2018-05-09,3.0,247046,1.121724,3
2,2018-05-10,3.0,247046,1.121724,0
3,2018-05-11,3.0,247046,1.043352,2
4,2018-05-12,3.0,247046,0.919101,0


In [78]:
df = pd.merge(final_result, raw_pred, how='left', on=list_keys)

In [79]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 601468 entries, 0 to 601467
Data columns (total 9 columns):
Date               601468 non-null datetime64[ns]
RDCKey             601468 non-null object
ProductKey         601468 non-null object
ypred              601468 non-null float32
model_2_ypred      601468 non-null float64
sales              601468 non-null int64
salesForecast      601468 non-null int64
ypred_raw          601468 non-null float64
salesForecast_y    601468 non-null int64
dtypes: datetime64[ns](1), float32(1), float64(2), int64(3), object(2)
memory usage: 43.6+ MB


In [80]:
df.head()

,Date,RDCKey,ProductKey,ypred,model_2_ypred,sales,salesForecast,ypred_raw,salesForecast_y
0,2018-05-08,3,247046,1.097878,1.288230,0,0,1.178358,0
1,2018-05-09,3,247046,1.048624,1.260161,3,3,1.121724,3
2,2018-05-10,3,247046,1.048624,1.278547,0,0,1.121724,0
3,2018-05-11,3,247046,0.994609,1.148104,2,2,1.043352,2
4,2018-05-12,3,247046,0.861172,0.862304,0,0,0.919101,0


In [81]:
df2 = df[df.Date<='2018-05-14']
df2['final_ensemble'] = 2 / ( 1 / df2.ypred_raw + 1 / df2.model_2_ypred)

df2['final_ensemble_2'] = (df2.ypred_raw + df2.model_2_ypred) / 2

#df2['final_ensemble_3'] = (0.303/(0.294+0.303)) * df2.ypred_raw + (0.294/(0.294+0.303)) * df2.model_2_ypred

print df2.final_ensemble.sum()
print np.sum(np.abs(df2.final_ensemble - df2.salesForecast)) / np.sum(df2.salesForecast)

print df2.final_ensemble_2.sum()
print np.sum(np.abs(df2.final_ensemble_2 - df2.salesForecast)) / np.sum(df2.salesForecast)

#print df2.final_ensemble_3.sum()
#print np.sum(np.abs(df2.final_ensemble_3 - df2.salesForecast)) / np.sum(df2.salesForecast)

print df2.salesForecast.sum()
print df2.ypred_raw.sum()
print df2.model_2_ypred.sum()
print "\n"
print np.sum(np.abs(df2.ypred_raw - df2.salesForecast)) / np.sum(df2.salesForecast)
print np.sum(np.abs(df2.model_2_ypred - df2.salesForecast)) / np.sum(df2.salesForecast)


235354.86474195844
0.7297488915064739
243576.01115476212
0.7425167440796409
251373
228522.22828100366
258629.7940285205


0.735336361197331
0.7753893398453598


/home/ubuntu/lijingjie/ljj/lib/python2.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/ubuntu/lijingjie/ljj/lib/python2.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [82]:
df2.Date.max()

Timestamp('2018-05-14 00:00:00')

In [83]:
print df2.Date.max()
print df2.Date.min()

2018-05-14 00:00:00
2018-05-08 00:00:00


In [85]:
df2 = df[(df.Date<='2018-05-14')]
df3 = df2.groupby(['RDCKey','ProductKey']).sum().reset_index()
df3['final_ensemble'] = 2 / ( 1 / df3.ypred_raw + 1 / df3.model_2_ypred)

df3['final_ensemble_2'] = (df3.ypred_raw + df3.model_2_ypred) / 2

#df3['final_ensemble_3'] = (0.303/(0.294+0.303)) * df2.ypred_raw + (0.294/(0.294+0.303)) * df2.model_2_ypred

print df3.final_ensemble.sum()
print np.sum(np.abs(df3.final_ensemble - df3.salesForecast)) / np.sum(df3.salesForecast)

print df3.final_ensemble_2.sum()
print np.sum(np.abs(df3.final_ensemble_2 - df3.salesForecast)) / np.sum(df3.salesForecast)

#print df2.final_ensemble_3.sum()
#print np.sum(np.abs(df2.final_ensemble_3 - df2.salesForecast)) / np.sum(df2.salesForecast)
print "\n"
print df3.salesForecast.sum()
print df3.ypred_raw.sum()
print df3.model_2_ypred.sum()
print "\n"
print np.sum(np.abs(df3.ypred_raw - df3.salesForecast)) / np.sum(df3.salesForecast)
print np.sum(np.abs(df3.model_2_ypred - df3.salesForecast)) / np.sum(df3.salesForecast)



238454.97984074915
0.39957562775186484
243576.01115476212
0.402832291184884


251373
228522.2282810037
258629.79402852056


0.40416607333153715
0.4332991815832395
